In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
seed=123

In [3]:
track_features = pd.read_csv('../../../data/v2.0/tracks_audio_features_v2.0.csv', sep='\t')
track_features.shape

(2245092, 14)

In [4]:
tracks = pd.read_csv('../../../data/original/tracks.csv', sep='\t')
tracks.shape
tracks.tail()
tracks = tracks[['tid', 'track_uri']]

(2262292, 6)

,tid,arid,alid,track_uri,track_name,duration_ms
2262287,2262287,93486,229637,spotify:track:2aOoiTTV0OR8DYxCk8o0JR,Forevermind - Pax217 Album Version,206400
2262288,2262288,93486,229637,spotify:track:1Uo65qTxnCg1N1X00lgcjr,Shalom - Pax217 Album Version,268266
2262289,2262289,93486,229637,spotify:track:5uEE5tii66I0cC7kZ7IMxE,Free To Be - Pax217 Album Version,334280
2262290,2262290,93486,229637,spotify:track:6A1RfnrMdxb24OYllzzTUX,Skwid - Pax217 Album Version,240453
2262291,2262291,295859,734683,spotify:track:5DvykFHB6utG6Hozpt5eNK,Running Thin,341933


In [5]:
tracks.tail()

,tid,track_uri
2262287,2262287,spotify:track:2aOoiTTV0OR8DYxCk8o0JR
2262288,2262288,spotify:track:1Uo65qTxnCg1N1X00lgcjr
2262289,2262289,spotify:track:5uEE5tii66I0cC7kZ7IMxE
2262290,2262290,spotify:track:6A1RfnrMdxb24OYllzzTUX
2262291,2262291,spotify:track:5DvykFHB6utG6Hozpt5eNK


In [ ]:
index_uri = track_features['uri']
track_features = track_features.drop(columns=['uri', 'duration_ms', 'key', 'time_signature'])
track_features['valence'].fillna(track_features['valence'].mean(), inplace=True)
track_features = track_features.astype(np.float32)
track_features['uri'] = index_uri
scaler = MinMaxScaler()
track_features['tempo'] = scaler.fit_transform(track_features.tempo.values.reshape(-1,1))
track_features['loudness'] = scaler.fit_transform(track_features.loudness.values.reshape(-1,1))

In [ ]:
track_features.tail()
tracks.tail()

In [ ]:
model = KMeans(n_clusters=5, n_jobs=-1, n_init=1)
track_features['cluster'] = model.fit_predict(track_features[track_features.columns[:-1]])
# audio_cols = track_features.columns[:-1]
# audio_cols
# cluster_means = track_features.groupby('cluster')[audio_cols].mean()
# cluster_means

In [ ]:
track_features = track_features[['uri', 'cluster']]
track_features.tail()

In [ ]:
track_features['track_uri'] = track_features['uri']
track_features.drop('uri', axis=1, inplace=True)

In [ ]:
a = pd.merge(tracks, track_features, on='track_uri',how='left')
a.tail()
a.shape

In [ ]:
a.isnull().sum()

In [ ]:
a.dropna(how='any', inplace=True)
a.shape
a.isnull().sum()

#### Cluster Analysis Elbow method

In [ ]:
scores = []
with open('../cluster_scores.txt', 'r') as f:
    for line in f:
        n = line.split(',')
        scores.append((n[0], n[1].strip()))

In [ ]:
plt.figure(figsize=(8,8))
plt.plot([int(l[0]) for l in scores],[float(l[1]) for l in scores])
#plt.xlim(0,18)
plt.grid(True)
plt.xlabel('K')
plt.ylabel('Error')